# message function

GCN 구현에 앞서 Graph Neural Network 의 핵심 개념인 Message Passing 을 짚고 넘어가보겠습니다. 

Message passing 은 총 3가지의 연산으로 이루어진 함수를 의미합니다. 크게 

① 정보를 전달하는 __message passing 파트__

② 앞서 ①에서의 정보를 바탕으로 hidden state의 갱신을 진행해주는 __update 파트__

③ ②에서 갱신된 hidden state의 값을 토대로 output 을 도출해내는 __readout 파트__로
로 나누어져있습니다.

In [ ]:
import numpy as np
from scipy.linalg import sqrtm 
from scipy.special import softmax
import networkx as nx
from networkx.algorithms.community.modularity_max import greedy_modularity_communities
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline

## Message Passing as Matrix Multiplication

Message Passing notation

$$x^{(k)}_i = \gamma^{(k)}(x^{(k-1)}_i, \square_{j\in N(i)} \phi^{(k)} (x^{(k-1)}_i, x^{(k-1)}_j, e_{j,i})$$

\
$$\square$$ 는 differentiable , permutation invariant 함수를 의미함. 예를들어, 합, 곱, 그리고 평균

$$\gamma \ and \ \phi$$ 는 differentiable function 을 의미함. 예를 들어, MLP 

In [ ]:
A = np.array(
    [[0,1,0,0,0],
     [1,0,1,0,0],
     [0,1,0,1,1],
     [0,0,1,0,0],
     [0,0,1,0,0]]
)

In [ ]:
feats = np.arange(A.shape[0]).reshape((-1,1))+1
# Adjacency 와 해당 노드의 feature을 곱해줌으로써 update 되는 과정
H = A @ feats
H

## normalization & scale the neighborhood information by neighborhood size

Scaling by neighborhood information size

In [ ]:
D = np.zeros(A.shape)
np.fill_diagonal(D, A.sum(axis=0))
print(D , '\n\n')

D_inv = np.linalg.inv(D)
print(D_inv , '\n\n')

H_avg = D_inv @ A @ feats
print(H_avg , '\n\n')

Normalized Adjacency Matrix

$$\hat{A} = \bar{D}^\frac{-1}{2} \bar{A} \bar{D}^\frac{-1}{2}$$

위 식을 구현하기 위해서 먼저 , $$\bar{A} = A + I$$ 를 구한다. 이 때 , A는 그래프간의 연결관계를 의미하는 Adjacency matrix 인접행렬 , I 는 Identity matrix 를 의미함.

이 때, Identity matrix(항등 행렬)을 포함해주는 이유는 self-connections 를 추가해주는 목적임.

In [ ]:
# nx.from_numpy_array 는 앞으로 matrix --> network 로 변환하는 용도로 활용도가 높은 함수이니 눈에 익히시는걸 추천드립니다.
g = nx.from_numpy_array(A)

A_mod = A + np.eye(g.number_of_nodes())

$$\bar{D}^\frac{-1}{2}$$ 를 도출하기 위해 다음과 같은 diagonal degree matrix 를 생성해야함.

$$(D)_{ij} = \delta_{i,j}\sum_kA_{i,k}$$

In [ ]:
# D for A_mod
D_mod = np.zeros_like(A_mod)
np.fill_diagonal(D_mod, A_mod.sum(axis=1).flatten())

# inverse square root of D
D_mod_invroot = np.linalg.inv(sqrtm(D_mod))
print(D_mod, "\n\n")

print(D_mod_invroot , "\n\n")

In [ ]:
node_labels = {i: i+1 for i in range(g.number_of_nodes())}
pos = nx.planar_layout(g)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
nx.draw(
    g, pos, with_labels=True,
    labels=node_labels,
    ax = ax,
    edge_color='gray',
    node_size=1500,
    font_size=30,
)

$$\hat{A} = \bar{D}^\frac{-1}{2} \bar{A} \bar{D}^\frac{-1}{2} $$

에서 $$\hat{A}$$를 구하기 위해선, 다음과 같은 연산이 필요함.

$$\hat{A}_{i,j} = \frac{\bar{A}_{i,j}}{{\bar{A}_{i,j}} \sqrt{\bar{d}_{i}\bar{d}_{j}}}$$



In [ ]:
A_hat = D_mod_invroot @ A_mod @ D_mod_invroot
print(A_hat)

In [ ]:
H = np.zeros((g.number_of_nodes(), 1))

In [ ]:
H[0,0] = 1 # initialize base results
H.flatten()

In [ ]:
iters = 10
results = [H.flatten()]
for i in range(iters):
  H = H * A_hat
  results.append(H.flatten())
  print(f"\n\n update iteration {i} results is \n\n " ,results[i])

In [ ]:
results

# GCN from scratch

[numpy](https://github.com/zjost/blog_code/blob/master/gcn_numpy/gcn_from_scratch.ipynb)

[torch](https://theaisummer.com/graph-convolutional-networks/)

[GCN paper](https://arxiv.org/abs/1609.02907)

[spectral graph](https://angeloyeo.github.io/2019/06/23/Fourier_Series.html) 

## numpy

In [ ]:
import numpy as np
from scipy.linalg import sqrtm
from scipy.special import softmax
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.algorithms.community.modularity_max import greedy_modularity_communities
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline
from IPython.display import HTML
import pandas as pd

In [ ]:
def draw_kkl(nx_G, 
             label_map,
             node_color,
             pos=None,
             **kwargs):
  fig, ax = plt.subplots(figsize=(15,10))
  if pos is None:
    pos = nx.spring_layout(nx_G, k=5/np.sqrt(nx_G.number_of_nodes()))

  nx.draw(
      nx_G,
      pos,
      with_labels=label_map is not None,
      labels = label_map,
      node_color = node_color,
      ax=ax,
      **kwargs
  )

### data download

In [ ]:
# (dataset)[https://snap.stanford.edu/data/email-Eu-core.html]
!wget https://snap.stanford.edu/data/email-Eu-core.txt.gz
!gzip -d email-Eu-core.txt.gz

In [ ]:
df = pd.read_csv('email-Eu-core.txt',header=None, sep='\s+', names= ['source','target'])
g = nx.from_pandas_edgelist(df)
g.number_of_nodes(), g.number_of_edges()

In [ ]:
!pip install python-louvain
from community import community_louvain
partition = community_louvain.best_partition(g)

In [ ]:
colors = np.zeros(g.number_of_nodes())

for index, com in enumerate(list(partition.values())):
  #print(index,com)
  colors[index] = com

n_classes = np.unique(colors).shape[0]
labels = np.eye(n_classes)[colors.astype(int)]

com_labels = nx.get_node_attributes(g,'community')

In [ ]:
_ = draw_kkl(g, None, colors, cmap='gist_rainbow', edge_color='gray')

[Random vs. Power-law Graphs](https://slidetodoc.com/peertopeer-and-social-networks-power-law-graphs-random/)


In [ ]:
degree_sequence = sorted((d for n, d in g.degree()), reverse=True)
degree_sequencedmax = max(degree_sequence)

fig = plt.figure('degree of a email graph', figsize=(15,8))
axgrid = fig.add_gridspec(5, 4)

ax0 = fig.add_subplot(axgrid[0:3, :])
Gcc = g.subgraph(sorted(nx.connected_components(g), key=len, reverse=True)[0])
pos = nx.spring_layout(Gcc, seed=20220806)
nx.draw_networkx_nodes(Gcc, pos, ax=ax0, node_size=20)
nx.draw_networkx_edges(Gcc, pos, ax=ax0, alpha=0.4)
ax0.set_title("Connected components of G")
ax0.set_axis_off()

ax1 = fig.add_subplot(axgrid[3:, :2])
ax1.plot(degree_sequence, "b-", marker="o")
ax1.set_title("Degree Rank Plot")
ax1.set_ylabel("Degree")
ax1.set_xlabel("Rank")

ax2 = fig.add_subplot(axgrid[3:, 2:])
ax2.bar(*np.unique(degree_sequence, return_counts=True))
ax2.set_title("Degree histogram")
ax2.set_xlabel("Degree")
ax2.set_ylabel("# of Nodes")

fig.tight_layout()
plt.show()

### Weight related functions

In [ ]:
# [intialize technique](https://yeomko.tistory.com/40)
def glorot_init(nin, nout):
  sd = np.sqrt(6.0 / (nin + nout))
  return np.random.uniform(-sd, sd , size=(nin, nout))

def xent(pred, labels):
  return -np.log(pred)[np.arange(pred.shape[0]), np.argmax(labels, axis=1)]

def norm_diff(dW, dW_approx):
  return np.linalg.norm(dW - dW_approx) / (np.linalg.norm(dW) + np.linalg.norm(dW_approx))

class GradDescentOptim():
  def __init__(self, lr, wd):
    self.lr = lr
    self.wd = wd
    self._y_pred = None
    self._y_true = None
    self._out = None
    self.bs = None
    self.train_nodes = None

  def __call__(self, y_pred, y_true, train_nodes=None):
    self.y_pred = y_pred
    self.y_true = y_true

    if train_nodes is None:
      self.train_nodes = np.arange(y_pred.shape[0])
    else:
      self.train_nodes = train_nodes

    self.bs = self.train_nodes.shape[0]

  @property
  def out(self):
    return self._out

  @out.setter
  def out(self, y):
    self._out = y

### GCN Layer 

In [ ]:
class GCNLayer():
  def __init__(self, n_inputs, n_outputs, activation=None, name=''):
    self.n_inputs = n_inputs
    self.n_outputs = n_outputs
    self.W = glorot_init(self.n_outputs, self.n_inputs)
    self.activation = activation
    self.name = name

  def __repr__(self):
    return f"GCN: W{'_'+self.name if self.name else ''} ({self.n_inputs}, ({self.n_outputs})"

  def forward(self, A, X, W=None):
    '''
    A is (bs,bs) adjacency matrix and X is (bs, D)
    where bs --> 'batch size' and D --> 'input feature length'
    '''
    self._A = A
    self._X = (A @ X).T # for calculating gradients. 

    if W is None:
      W = self.W
    
    H = W @ self._X # (h, D) * (D , bs) -> (h, bs)
    if self.activation is not None:
      H = self.activation(H)
    self._H = H # (h, bs)
    return self._H.T # (bs, h)

  def backward(self, optim, update=True):
    dtanh = 1 - np.asarray(self._H.T) ** 2
    d2 = np.multiply(optim.out, dtanh) # (bs, out_dim) * element_wise * (bs, out_dim)

    self.grad = self._A @ d2 @ self.W
    optim.out = self.grad

    dW = np.asarray(d2.T @ self._X.T) / optim.bs # (out_dim, bs) * (bs, D) --> (out_dim, D) 
    dW_wd = self.W * optim.wd / optim.bs # weight decay update

    if update:
      self.W -= (dW + dW_wd) * optim.lr
    
    return dW + dW_wd

### Activation function

In [ ]:
class SoftmaxLayer():
    def __init__(self, n_inputs, n_outputs, name=''):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = glorot_init(self.n_outputs, self.n_inputs)
        self.b = np.zeros((self.n_outputs, 1))
        self.name = name
        self._X = None # Used to calculate gradients
        
    def __repr__(self):
        return f"Softmax: W{'_'+self.name if self.name else ''} ({self.n_inputs}, {self.n_outputs})"
    
    def shift(self, proj):
        shiftx = proj - np.max(proj, axis=0, keepdims=True)
        exps = np.exp(shiftx)
        return exps / np.sum(exps, axis=0, keepdims=True)
        
    def forward(self, X, W=None, b=None):
        """Compute the softmax of vector x in a numerically stable way.
        
        X is assumed to be (bs, h)
        """
        self._X = X.T
        if W is None:
            W = self.W
        if b is None:
            b = self.b

        proj = np.asarray(W @ self._X) + b # (out, h)*(h, bs) = (out, bs)
        return self.shift(proj).T # (bs, out)
    
    def backward(self, optim, update=True):
        # should take in optimizer, update its own parameters and update the optimizer's "out"
        # Build mask on loss
        train_mask = np.zeros(optim.y_pred.shape[0])
        train_mask[optim.train_nodes] = 1
        train_mask = train_mask.reshape((-1, 1))
        
        # derivative of loss w.r.t. activation (pre-softmax)
        d1 = np.asarray((optim.y_pred - optim.y_true)) # (bs, out_dim)
        d1 = np.multiply(d1, train_mask) # (bs, out_dim) with loss of non-train nodes set to zero
        
        self.grad = d1 @ self.W # (bs, out_dim)*(out_dim, in_dim) = (bs, in_dim)
        optim.out = self.grad
        
        dW = (d1.T @ self._X.T) / optim.bs  # (out_dim, bs)*(bs, in_dim) -> (out_dim, in_dim)
        db = d1.T.sum(axis=1, keepdims=True) / optim.bs # (out_dim, 1)
                
        dW_wd = self.W * optim.wd / optim.bs # weight decay update
        
        if update:   
            self.W -= (dW + dW_wd) * optim.lr
            self.b -= db.reshape(self.b.shape) * optim.lr
        
        return dW + dW_wd, db.reshape(self.b.shape)

In [ ]:
gcn1 = GCNLayer(g.number_of_nodes(), 2, activation=np.tanh, name='1')
sm1 = SoftmaxLayer(2, n_classes, "SM")
opt = GradDescentOptim(lr=0, wd=1.)

In [ ]:
gcn1_out = gcn1.forward(A_hat, X)
opt(sm1.forward(gcn1_out), labels)

In [ ]:
def get_grads(inputs, layer, argname, labels, eps=1e-4, wd=0):
    cp = getattr(layer, argname).copy()
    cp_flat = np.asarray(cp).flatten()
    grads = np.zeros_like(cp_flat)
    n_parms = cp_flat.shape[0]
    for i, theta in enumerate(cp_flat):
        #print(f"Parm {argname}_{i}")
        theta_cp = theta
        
        # J(theta + eps)
        cp_flat[i] = theta + eps
        cp_tmp = cp_flat.reshape(cp.shape)
        predp = layer.forward(*inputs, **{argname: cp_tmp})
        wd_term = wd/2*(cp_flat**2).sum() / labels.shape[0]
        #print(wd_term)
        Jp = xent(predp, labels).mean() + wd_term
        
        # J(theta - eps)
        cp_flat[i] = theta - eps
        cp_tmp = cp_flat.reshape(cp.shape)
        predm = layer.forward(*inputs, **{argname: cp_tmp})
        wd_term = wd/2*(cp_flat**2).sum() / labels.shape[0]
        #print(wd_term)
        Jm = xent(predm, labels).mean() + wd_term
        
        # grad
        grads[i] = ((Jp - Jm) / (2*eps))
        
        # Back to normal
        cp_flat[i] = theta

    return grads.reshape(cp.shape)

In [ ]:
dW_approx = get_grads((gcn1_out,), sm1, "W", labels, eps=1e-4, wd=opt.wd)
db_approx = get_grads((gcn1_out,), sm1, "b", labels, eps=1e-4, wd=opt.wd)

In [ ]:
# Get gradients on Linear Softmax layer
dW, db = sm1.backward(opt, update=False)

In [ ]:
assert norm_diff(dW, dW_approx) < 1e-7
assert norm_diff(db, db_approx) < 1e-7

In [ ]:
def get_gcn_grads(inputs, gcn, sm_layer, labels, eps=1e-4, wd=0):
    cp = gcn.W.copy()
    cp_flat = np.asarray(cp).flatten()
    grads = np.zeros_like(cp_flat)
    n_parms = cp_flat.shape[0]
    for i, theta in enumerate(cp_flat):
        theta_cp = theta
        
        # J(theta + eps)
        cp_flat[i] = theta + eps
        cp_tmp = cp_flat.reshape(cp.shape)
        pred = sm_layer.forward(gcn.forward(*inputs, W=cp_tmp))
        w2 = (cp_flat**2).sum()+(sm_layer.W.flatten()**2).sum()
        Jp = xent(pred, labels).mean() + wd/(2*labels.shape[0])*w2
        
        # J(theta - eps)
        cp_flat[i] = theta - eps
        cp_tmp = cp_flat.reshape(cp.shape)
        pred = sm_layer.forward(gcn.forward(*inputs, W=cp_tmp))
        w2 = (cp_flat**2).sum()+(sm_layer.W.flatten()**2).sum()
        Jm = xent(pred, labels).mean() + wd/(2*labels.shape[0])*w2
        
        # grad
        grads[i] = ((Jp - Jm) / (2*eps))
        
        # Back to normal
        cp_flat[i] = theta

    return grads.reshape(cp.shape)

In [ ]:
dW2 = gcn1.backward(opt, update=False)
dW2_approx = get_gcn_grads((A_hat, X), gcn1, sm1, labels, eps=1e-4, wd=opt.wd)
assert norm_diff(dW2, dW2_approx) < 1e-7

In [ ]:
def get_gcn_input_grads(A_hat, X, gcn, sm_layer, labels, eps=1e-4):
    cp = X.copy()
    cp_flat = np.asarray(cp).flatten()
    grads = np.zeros_like(cp_flat)
    n_parms = cp_flat.shape[0]
    for i, x in enumerate(cp_flat):
        x_cp = x
        
        # J(theta + eps)
        cp_flat[i] = x + eps
        cp_tmp = cp_flat.reshape(cp.shape)
        pred = sm_layer.forward(gcn.forward(A_hat, cp_tmp))
        Jp = xent(pred, labels).mean()
        
        # J(theta - eps)
        cp_flat[i] = x - eps
        cp_tmp = cp_flat.reshape(cp.shape)
        pred = sm_layer.forward(gcn.forward(A_hat, cp_tmp))
        Jm = xent(pred, labels).mean()
        
        # grad
        grads[i] = ((Jp - Jm) / (2*eps))
        
        # Back to normal
        cp_flat[i] = x

    return grads.reshape(cp.shape)

In [ ]:
dX_approx = get_gcn_input_grads(A_hat, X, gcn1, sm1, labels, eps=1e-4)
assert norm_diff(gcn1.grad/A_hat.shape[0], dX_approx) < 1e-7

### GNN Model

In [ ]:
class GCNNetwork():
    def __init__(self, n_inputs, n_outputs, n_layers, hidden_sizes, activation, seed=0):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.n_layers = n_layers
        self.hidden_sizes = hidden_sizes
        self.activation = activation
        
        np.random.seed(seed)
        
        self.layers = list()
        # Input layer
        gcn_in = GCNLayer(n_inputs, hidden_sizes[0], activation, name='in')
        self.layers.append(gcn_in)
        
        # Hidden layers
        for layer in range(n_layers):
            gcn = GCNLayer(self.layers[-1].W.shape[0], hidden_sizes[layer], activation, name=f'h{layer}')
            self.layers.append(gcn)
            
        # Output layer
        sm_out = SoftmaxLayer(hidden_sizes[-1], n_outputs, name='sm')
        self.layers.append(sm_out)
        
    def __repr__(self):
        return '\n'.join([str(l) for l in self.layers])
    
    def embedding(self, A, X):
        # Loop through all GCN layers
        H = X
        for layer in self.layers[:-1]:
            H = layer.forward(A, H)
        return np.asarray(H)
    
    def forward(self, A, X):
        # GCN layers
        H = self.embedding(A, X)
        
        # Softmax
        p = self.layers[-1].forward(H)
        
        return np.asarray(p)

In [ ]:
gcn_model = GCNNetwork(
    n_inputs=g.number_of_nodes(), 
    n_outputs=n_classes, 
    n_layers=2,
    hidden_sizes=[16, 2], 
    activation=np.tanh,
    seed=100,
)
gcn_model

In [ ]:
y_pred = gcn_model.forward(A_hat, X)
embed = gcn_model.embedding(A_hat, X)
xent(y_pred, labels).mean()

### Training

In [ ]:

train_nodes = np.array([])
test_nodes = np.array([i for i in range(labels.shape[0]) if i not in train_nodes])
opt2 = GradDescentOptim(lr=2e-2, wd=2.5e-2)

embeds = list()
accs = list()
train_losses = list()
test_losses = list()

loss_min = 1e6
es_iters = 0
es_steps = 50
lr_rate_ramp = 0 #-0.05
lr_ramp_steps = 1000

for epoch in range(15000):
    
    y_pred = gcn_model.forward(A_hat, X)

    opt2(y_pred, labels, train_nodes)
    
    if ((epoch+1) % lr_ramp_steps) == 0:
        opt2.lr *= 1+lr_rate_ramp
        print(f"LR set to {opt2.lr:.4f}")

    for layer in reversed(gcn_model.layers):
        layer.backward(opt2, update=True)
        
    embeds.append(gcn_model.embedding(A_hat, X))
    # Accuracy for non-training nodes
    acc = (np.argmax(y_pred, axis=1) == np.argmax(labels, axis=1))[
        [i for i in range(labels.shape[0]) if i not in train_nodes]
    ]
    accs.append(acc.mean())
    
    loss = xent(y_pred, labels)
    loss_train = loss[train_nodes].mean()
    loss_test = loss[test_nodes].mean()
    
    train_losses.append(loss_train)
    test_losses.append(loss_test)
    
    if loss_test < loss_min:
        loss_min = loss_test
        es_iters = 0
    else:
        es_iters += 1
        
    if es_iters > es_steps:
        print("Early stopping!")
        break
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch+1}, Train Loss: {loss_train:.3f}, Test Loss: {loss_test:.3f}")
        
train_losses = np.array(train_losses)
test_losses = np.array(test_losses)

fig, ax = plt.subplots()
ax.plot(np.log10(train_losses), label='Train')
ax.plot(np.log10(test_losses), label='Test')
ax.legend()
ax.grid()

## torch

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse as sparse

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import gc


### data download

In [ ]:
# (dataset)[https://snap.stanford.edu/data/email-Eu-core.html]
!wget https://snap.stanford.edu/data/email-Eu-core.txt.gz
!gzip -d email-Eu-core.txt.gz -f

# network formatting
df = pd.read_csv('email-Eu-core.txt',header=None, sep='\s+', names= ['source','target'])
g = nx.from_pandas_edgelist(df)
print(f'node counts ; {g.number_of_nodes()}, edge counts ; {g.number_of_edges()}')

# tensor formatting
adj_g = torch.Tensor(nx.to_numpy_array(g))
print(adj_g.size())

### matrix handling

In [ ]:
def calc_degree_matrix(a):
    '''
    a ; adjacency matrix
    '''
    return torch.diag(a.sum(axis=-1))

def create_graph_lapl(a):
    return calc_degree_matrix(a)-a

def calc_degree_matrix_norm(a):
    return torch.diag(torch.pow(a.sum(axis=-1),-0.5))

def create_graph_lapl_norm(a):
    size = a.shape[-1]
    D_norm = calc_degree_matrix_norm(a)
    L_norm = torch.ones(size) - (D_norm @ a @ D_norm )
    return L_norm

### Chebyshev approximation for the Laplacian powers

- Spectral concept -> Laplacian eigenvectors -> SVD of L -> approximation Chebyshev expansion



- The Chebyshev approximation is a __recurrent expansion that uses a matrix to estimate the matrix in any given power K__ , thus avoding the K square matrix mulitplications.



[한국어자료](https://datalabbit.tistory.com/26)

In [ ]:
# torch.eig ; https://pytorch.org/docs/stable/generated/torch.eig.html
def find_eigmax(L):
    '''
    L ; Laplaican matrix
    '''
    with torch.no_grad():
        e1, _ = torch.eig(L, eigenvectors=False)
        return torch.max(e1[:, 0]).item()

def chebyshev_Lapl(X, Lapl, thetas, order):
    list_powers = []
    nodes = Lapl.shape[0]

    T0 = X.float()

    eigmax = find_eigmax(Lapl)
    L_rescaled = (2 * Lapl / eigmax) - torch.eye(nodes)

    y = T0 * thetas[0]
    list_powers.append(y)
    T1 = torch.matmul(L_rescaled, T0)
    list_powers.append(T1 * thetas[1])

    # Computation of: T_k = 2*L_rescaled*T_k-1  -  T_k-2
    for k in range(2, order):
      T2 = 2 * torch.matmul(L_rescaled, T1) - T0
      list_powers.append((T2 * thetas[k]))
      T0, T1 = T1, T2
    y_out = torch.stack(list_powers, dim=-1)
    # the powers may be summed or concatenated. i use concatenation here
    y_out = y_out.view(nodes, -1) # -1 = order* features_of_signal
    return y_out

In [ ]:
features = 3
out_features = 50
L = create_graph_lapl_norm(adj_g)
x = torch.rand(adj_g.size(-1), features)
power_order = 4 # p-hops
thetas = nn.Parameter(torch.rand(4))

In [ ]:
out = chebyshev_Lapl(x,L,thetas,power_order)
print('cheb approx out powers concatenated:', out.shape)
linear = nn.Linear(4*3, out_features)
layer_out = linear(out)
print(f'Layers output; {layer_out.shape}')

### GCN Layer

In [ ]:
!pip install torchnet

In [ ]:
import numpy as np

import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
import torchnet as tnt

import os
import networkx as nx


def device_as(x,y):
  return x.to(y.device)

# tensor operationa now support batched inputs
def calc_degree_matrix_norm(a):
  return torch.diag_embed(torch.pow(a.sum(dim=-1),-0.5))

def create_graph_lapl_norm(a):
  size = a.shape[-1]
  a +=  device_as(torch.eye(size),a)
  D_norm = calc_degree_matrix_norm(a)
  L_norm = torch.bmm( torch.bmm(D_norm, a) , D_norm )
  return L_norm

class GCN_AISUMMER(nn.Module):
    """
    A simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    def __init__(self, in_features, out_features, bias=True):
      super().__init__()
      self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, X, A):
        """
        A: adjecency matrix
        X: graph signal
        """
        L = create_graph_lapl_norm(A)
        x = self.linear(X)
        return torch.bmm(L, x)

### data download for graph classification task

dataset preprcessing 활용되는 라이브러리 

[torchnet](https://github.com/pytorch/tnt?fbclid=IwAR0upau00LEWwoBdwhqY5QNT3-L8IUA61mEepkGyAAuKBBEkekfMio-TRNI#getting-started)

MUTAH dataset description ; 

graphs are used to represent chemical compounds, where vertices stand for atoms and are labeled by the atom type (represented by one-hot encoding), while edges between vertices represent bonds between the corresponding atoms. It includes 188 samples of chemical compounds with 7 discrete node labels.

In [ ]:
!wget https://ls11-www.cs.tu-dortmund.de/people/morris/graphkerneldatasets/MUTAG.zip
!unzip MUTAG.zip
!pip install torchnet

### graph utils and data loading

In [ ]:
def indices_to_one_hot(number, nb_classes, label_dummy=-1):
    """Convert an iterable of indices to one-hot encoded labels."""
    if number == label_dummy:
        return np.zeros(nb_classes)
    else:
        return np.eye(nb_classes)[number]

def get_graph_signal(nx_graph):
  d = dict((k, v) for k, v in nx_graph.nodes.items())
  x = []
  invd = {}
  j = 0
  for k, v in d.items():
      x.append(v['attr_dict'])
      invd[k] = j
      j = j + 1
  return np.array(x)


def load_data(path, ds_name, use_node_labels=True, max_node_label=10):
    node2graph = {}
    Gs = []
    data = []
    dataset_graph_indicator = f"{ds_name}_graph_indicator.txt"
    dataset_adj = f"{ds_name}_A.txt"
    dataset_node_labels = f"{ds_name}_node_labels.txt"
    dataset_graph_labels = f"{ds_name}_graph_labels.txt"

    path_graph_indicator = os.path.join(path,dataset_graph_indicator)
    path_adj = os.path.join(path,dataset_adj)
    path_node_lab = os.path.join(path,dataset_node_labels)
    path_labels = os.path.join(path,dataset_graph_labels)


    with open(path_graph_indicator, "r") as f:
        c = 1
        for line in f:
            node2graph[c] = int(line[:-1])
            if not node2graph[c] == len(Gs):
                Gs.append(nx.Graph())
            Gs[-1].add_node(c)
            c += 1

    with open(path_adj, "r") as f:
        for line in f:
            edge = line[:-1].split(",")
            edge[1] = edge[1].replace(" ", "")
            Gs[node2graph[int(edge[0])] - 1].add_edge(int(edge[0]), int(edge[1]))

    if use_node_labels:
      with open(path_node_lab, "r") as f:
        c = 1
        for line in f:
          node_label = indices_to_one_hot(int(line[:-1]), max_node_label)
          Gs[node2graph[c] - 1].add_node(c, attr_dict=node_label)
          c += 1

    labels = []
    with open(path_labels, "r") as f:
        for line in f:
            labels.append(int(line[:-1]))

    return list(zip(Gs, labels)) 

def create_loaders(dataset, batch_size, split_id, offset=-1):
    train_dataset = dataset[:split_id]
    val_dataset = dataset[split_id:]
    return to_pytorch_dataset(train_dataset, offset,batch_size), to_pytorch_dataset(val_dataset, offset,batch_size)

def to_pytorch_dataset(dataset, label_offset=0, batch_size=1):
  list_set = []
  for graph, label in dataset:
    F, G = get_graph_signal(graph), nx.to_numpy_matrix(graph)
    numOfNodes = G.shape[0]
    F_tensor = torch.from_numpy(F).float()
    G_tensor = torch.from_numpy(G).float()

    # fix labels to zero-indexing
    if label == -1:
      label = 0
    
    label += label_offset
    
    list_set.append(tuple((F_tensor, G_tensor, label)))

  dataset_tnt = tnt.dataset.ListDataset(list_set)
  data_loader = torch.utils.data.DataLoader(dataset_tnt, shuffle=True, batch_size=batch_size)
  return data_loader



dataset = load_data(path='./MUTAG/', ds_name='MUTAG',
                  use_node_labels=True, max_node_label=7)
train_dataset, val_dataset = create_loaders(dataset, batch_size=1, split_id=150, offset=0)
print('Data are ready')

### GNN model 

In [ ]:
class GNN(nn.Module):
  def __init__(self,
                    in_features = 7,
                    hidden_dim = 64,
                    classes = 2,
                    dropout = 0.5):
    super(GNN, self).__init__()

    self.conv1 = GCN_AISUMMER(in_features, hidden_dim)
    self.conv2 = GCN_AISUMMER(hidden_dim, hidden_dim)
    self.conv3 = GCN_AISUMMER(hidden_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, classes)
    self.dropout = dropout

  def forward(self, x,A):
    x = self.conv1(x, A)
    x = F.relu(x)
    x = self.conv2(x, A)
    x = F.relu(x)
    x = self.conv3(x, A)
    x = F.dropout(x, p=self.dropout, training=self.training)
    # aggregate node embeddings
    x = x.mean(dim=1)
    # final classification layer
    return self.fc(x)



### Training

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'Training on {device}')
model = GNN(in_features = 7,
                hidden_dim = 128,
                classes = 2).to(device)

optimizer= torch.optim.SGD(model.parameters(), lr=0.01)

def train(train_loader):
    model.train()

    for data in train_loader: 
      optimizer.zero_grad()  
      X, A, labels = data
      X, A, labels = X.to(device), A.to(device), labels.to(device)  
      # Forward pass.
      out = model(X, A)  
      # Compute the graph classification loss.
      loss = criterion(out, labels) 
      # Calculate gradients.
      loss.backward()  
      # Updates the models parameters
      optimizer.step() 

def test(loader):
  model.eval()
  correct = 0
  for data in loader:
    X,A, labels = data
    X,A, labels = X.cuda(), A.cuda(), labels.cuda()  
    # Forward pass.
    out = model(X, A)  
    # Take the index of the class with the highest probability.
    pred = out.argmax(dim=1) 
    # Compare with ground-truth labels.
    correct += int((pred == labels).sum()) 
  return correct / len(loader.dataset)  


# main code :)

best_val = -1
for epoch in range(1, 241):
    train(train_dataset)
    train_acc = test(train_dataset)
    val_acc = test(val_dataset)
    if val_acc>best_val:
      best_val = val_acc
      epoch_best = epoch
    
    if epoch%10==0:
      print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f} || Best Val Score: {best_val:.4f} (Epoch {epoch_best:03d}) ')


# Correct & Smooth 로 성능 향상해보기.

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

import torch_geometric
from torch_geometric.nn import MLP, CorrectAndSmooth

In [ ]:
import os.path as osp

import torch
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset

import torch_geometric.transforms as T
from torch_geometric.nn import MLP, CorrectAndSmooth

root = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'OGB')
dataset = PygNodePropPredDataset('ogbn-products', root,
                                 transform=T.ToSparseTensor())
evaluator = Evaluator(name='ogbn-products')
split_idx = dataset.get_idx_split()
data = dataset[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP([dataset.num_features, 200, 200, dataset.num_classes], dropout=0.5,
            norm="batch_norm", act_first=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

x, y = data.x.to(device), data.y.to(device)
train_idx = split_idx['train'].to(device)
val_idx = split_idx['valid'].to(device)
test_idx = split_idx['test'].to(device)
x_train, y_train = x[train_idx], y[train_idx]


def train():
    model.train()
    optimizer.zero_grad()
    out = model(x_train)
    loss = criterion(out, y_train.view(-1))
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(out=None):
    model.eval()
    out = model(x) if out is None else out
    pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': y[train_idx],
        'y_pred': pred[train_idx]
    })['acc']
    val_acc = evaluator.eval({
        'y_true': y[val_idx],
        'y_pred': pred[val_idx]
    })['acc']
    test_acc = evaluator.eval({
        'y_true': y[test_idx],
        'y_pred': pred[test_idx]
    })['acc']
    return train_acc, val_acc, test_acc, out


best_val_acc = 0
for epoch in range(1, 301):
    loss = train()
    train_acc, val_acc, test_acc, out = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        y_soft = out.softmax(dim=-1)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

adj_t = data.adj_t.to(device)
deg = adj_t.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow_(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
DAD = deg_inv_sqrt.view(-1, 1) * adj_t * deg_inv_sqrt.view(1, -1)
DA = deg_inv_sqrt.view(-1, 1) * deg_inv_sqrt.view(-1, 1) * adj_t

post = CorrectAndSmooth(num_correction_layers=50, correction_alpha=1.0,
                        num_smoothing_layers=50, smoothing_alpha=0.8,
                        autoscale=False, scale=20.)

print('Correct and smooth...')
y_soft = post.correct(y_soft, y_train, train_idx, DAD)
y_soft = post.smooth(y_soft, y_train, train_idx, DA)
print('Done!')
train_acc, val_acc, test_acc, _ = test(y_soft)
print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

### Level up

 1. Spectral Filtering 관점을 graph transformer 의 positional encoding에 어떻게 적용할 수 있을지 고민해보기

 2. [labelSmoothing](https://ratsgo.github.io/insight-notes/docs/interpretable/smoothing) & [C&S](https://seungwooham.tistory.com/entry/CS224w-05-Message-Passing-and-Node-Classification) 관점 이해한 후 , 적용해보기

 3. Feature engineering 측면으로 network science 를 어떻게 적용할 수 있을지 고민해보기.

 4. Numpy 섹션에서의 community detection 으로 나온 결과와 실제 community label (ground truth) 를 비교해보고 각 community detection 알고리즘 적용 후 왜 그런 결과가 도출됬는지에 대해 고민해보기.

 5. over smoothing & over squashing 현상에 대한 이해, 그리고 왜 그런현상이 발생하는지에 대해 graph distribution (그래프 특성)과 연관지어 고민해보기.

# Level up 에 관하여 궁금한 점 있으실 시 , 정이태 jeongiitae6@gmail.com 로 연락주세요! :)